### 02. Neural Network classification with PyTorch
Classification is a problem of predicting whether something is one thing or another (there can be multiple things as the options).

#### 1. Make classification data and get it ready

In [ ]:
import sklearn
from sklearn.datasets import make_circles

# Make 1000 samples

n_samples = 1000

# Create circles
x,y = make_circles(n_samples, noise=0.03, random_state=42)

In [ ]:
len(x), len(y)

In [ ]:
print(f'First 5 samples of x:\n {x[:5]}')
print(f'First 5 samples of y:\n {y[:5]}')

In [ ]:
# Make DataFrame of circle data
import pandas as pd

circles = pd.DataFrame({"X1": x[:, 0], "X2": x[:, 1], "label": y})
circles.head(10)

In [ ]:
# Visualize, # Visualize, # Visualize
import matplotlib.pyplot as plt
plt.scatter(x=x[:,0],
            y=x[:,1],
            c=y,
            cmap=plt.cm.RdYlBu)

### 1.1 Check input and output shapes

In [ ]:
x.shape, y.shape

In [ ]:
x

In [ ]:
# View the first example of features and labels
x_sample = x[0]
y_sample = y[0]

print(f"Values for one sample of x: {x_sample} and the same for y: {y_sample}")
print(f"Shapes for one sample of x: {x_sample.shape} and the same for y: {y_sample.shape}")

In [ ]:
type(x)

In [ ]:
# Turn data into tensors
import torch
x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

x[:5], y[:5]

In [ ]:
type(x), x.dtype, y.dtype

In [ ]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
len(x_train), len(x_test), len(y_train), len(y_test)